In [1]:
import nltk
import difflib
import time
import gc
import itertools
import multiprocessing
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold

from models_utils_fe import *
from models_utils_skf import *

In [2]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
feats_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/uncleaned/'
oof_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/OOF_preds/train/'

X_train = pd.read_pickle('Xtrain_866BestColsDropped.pkl')
mlp = pd.read_pickle(oof_src + 'train_preds_MLP_1sttry.pkl')
X_train = pd.concat([X_train, mlp], axis = 1)
X_train = X_train.astype('float32')
print(X_train.shape)

xgb_feats = pd.read_csv(feats_src + '/the_1owl/owl_train.csv')
y_train = xgb_feats[['is_duplicate']]

del xgb_feats
gc.collect()

(404290, 866)


21

In [3]:
lgb_params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : {'binary_logloss'},
    'learning_rate' : 0.03,
    'feature_fraction' : 0.51,
    'bagging_fraction': 0.9,
    'bagging_freq': 100,
    'num_leaves' : 255,
    'max_depth': 8,
    'min_data_in_leaf': 23,
    'subsample': 0.8,
    'colsample_bytree': 0.41,
    'silent': 1,
    'random_state': 1337,
    'verbose': 1,
    'nthread': 9,
}

xgb_params = {
    'seed': 1337,
    'colsample_bytree': 0.44,
    'silent': 1,
    'subsample': 0.82,
    'eta': 0.02,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 7,
    'min_child_weight': 22,
    'nthread': 4,
    'tree_method': 'hist',
    }

* Columns dropped
```python
[100]	valid_0's binary_logloss: 0.211293
[200]	valid_0's binary_logloss: 0.189018
[300]	valid_0's binary_logloss: 0.185001
[400]	valid_0's binary_logloss: 0.182913
[500]	valid_0's binary_logloss: 0.181672
[600]	valid_0's binary_logloss: 0.180901
[700]	valid_0's binary_logloss: 0.180319
[800]	valid_0's binary_logloss: 0.179623
[900]	valid_0's binary_logloss: 0.179295
[1000]	valid_0's binary_logloss: 0.179195
[1100]	valid_0's binary_logloss: 0.179099
[1200]	valid_0's binary_logloss: 0.179294
Early stopping, best iteration is:
[1074]	valid_0's binary_logloss: 0.179023
Start predicting...
Final score for fold 1 : 0.179031600541 
```

In [ ]:
lgb_foldrun(X_train, y_train, lgb_params, 'LGB_MLPadded')

In [ ]:
gbm = xgb.Booster(model_file = 'saved_models/XGB/XGB_10SKF_FredFeatsGRU_loss0.17917_fold1.txt')
dtrain = xgb.DMatrix(X_train, label = y_train)

mapper = {'f{0}'.format(i): v for i, v in enumerate(dtrain.feature_names)}
importance = {mapper[k]: v for k, v in gbm.get_fscore().items()}
importance = sorted(importance.items(), key=lambda x:x[1], reverse=True)[:20]

df_importance = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_importance['fscore'] = df_importance['fscore'] / df_importance['fscore'].sum()

plt.figure()
df_importance.plot()
df_importance.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(10, 18))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')